## single-source shortest path (SSSP)

In [1]:
"""
What is Single-Source Shortest-Path (SSSP) Algorithm:
    An algorithm that finds the shortest paths from a source vertex to all other vertices in a graph. 
    It works on directed or undirected, weighted or unweighted graphs, aiming to find the path with the minimum total weight (cost or distance)

Shortest Path Algorithms:
    - BFS:
        For unweighted graphs or unit-weighted edges. Time: O(V + E).
    - Dijkstra:
        For graphs with non-negative weights, using a priority queue. Time: O((V + E) log V).
    - Bellman-Ford:
        Handles negative weights and detects negative weight cycles. Time: O(V * E).
    - Floyd-Warshall (Not SSSP):
        For solving all-pairs shortest paths (APSP) problem. Time: O(V^3).
    - A*:
        Heuristic-based algorithm for navigation and AI, combining path cost and heuristic. Time: Depends on the heuristic.

Applications:
    - Computer Networks: Used in routing protocols like OSPF and RIP.
    - Navigation Systems: GPS and mapping for route optimization.
    - Social Networks: Finding shortest connections between users.
    - Operations Research: Optimizing logistics and transportation.
    - Robotics: Pathfinding for autonomous systems.
"""

pass

## BFS

## Dijkstra

#### Pseudocode

In [2]:
"""
Pseudocode:
    1. Initialize:
        - Set all vertex distance to infinity, and the source vertex to 0.
        - Use a priority queue: priority_queue = [(0, src)]
        - Create a `visited` set.

    2. While the priority queue is not empty:
        - Extract the vertex `current_vertex` with the smallest distance.
        - Skip if already visited; otherwise, mark as visited.

    3. Edge Relaxation:
        -For each neighbor of `current_vertex`:
            - Calculate `new_distance = distance[current_vertex] + edge_weight`.
            - If `new_distance` is smaller, update `distance[neighbor]` and add it to the queue.

    4. Return:
        return `distance` 

links:
    - https://www.youtube.com/watch?v=EFg3u_E6eHU

Complexity:
    - Time: O((V + E) log V)
    - Space: O(V + E)
"""

pass

#### Code 

In [3]:
import heapq


def dijkstra(graph: dict, source):

    distance = {v: float("inf") for v in graph}  # Every vertex distance Infinity
    distance[source] = 0
    min_heap = [(0, source)]  # (distance, source)
    visited = set()

    while min_heap:
        current_distance, current_vertex = heapq.heappop(min_heap)

        if current_vertex in visited:
            continue

        visited.add(current_vertex)

        for neighbor, weight in graph[current_vertex]:
            # neighbor_distance = distance[current_vertex] + weight
            neighbor_distance = current_distance + weight

            # Edge Relaxation
            if neighbor_distance < distance[neighbor]:  #  d[u] + c(u, v) < d[v]
                distance[neighbor] = neighbor_distance
                heapq.heappush(min_heap, (neighbor_distance, neighbor))

    return distance

In [4]:
graph = {
    "A": [("D", 1)],
    "B": [("C", 7), ("D", 2)],
    "C": [("B", 7), ("D", 4)],
    "D": [("A", 1), ("B", 2), ("C", 4)],
}

dijkstra(graph, "A")

{'A': 0, 'B': 3, 'C': 5, 'D': 1}

#### Dijkstra v2: Shortest Path along with Path Reconstruction

In [5]:
import heapq


def dijkstra_v2(graph: dict, source):
    visited = set()
    parents = {source: None}  # Track the parents

    distance = {v: float("inf") for v in graph}  # Every vertex distance Infinity
    distance[source] = 0
    min_heap = [(0, source)]  # (distance, source)

    while min_heap:
        current_distance, current_vertex = heapq.heappop(min_heap)

        if current_vertex in visited:
            continue

        visited.add(current_vertex)

        for neighbor, weight in graph[current_vertex]:
            # neighbor_distance = distance[current_vertex] + weight
            neighbor_distance = current_distance + weight

            # Edge Relaxation
            if neighbor_distance < distance[neighbor]:  #  d[u] + c(u, v) < d[v]
                distance[neighbor] = neighbor_distance
                heapq.heappush(min_heap, (neighbor_distance, neighbor))
                parents[neighbor] = current_vertex

    return distance, parents


def reconstruct_path(parents: dict, src, dest):
    path = []
    current = dest
    while current is not None:
        path.append(current)
        if current == src:
            break
        current = parents.get(current)

    if path[-1] != src:  # No path exists
        return []

    return path[::-1]  # Reverse the path

In [6]:
graph = {
    "A": [("B", 1), ("D", 8)],
    "B": [("A", 1), ("C", 2), ("D", 6)],
    "C": [("B", 2), ("D", 3)],
    "D": [("A", 8), ("B", 6), ("C", 3)],
}
distance, parent = dijkstra_v2(graph, "A")
path = reconstruct_path(parent, "A", "D")

print(f"{distance = }")
print(f"{parent = }")
print(f"Path: {' -> '.join(path)}")

distance = {'A': 0, 'B': 1, 'C': 3, 'D': 6}
parent = {'A': None, 'B': 'A', 'D': 'C', 'C': 'B'}
Path: A -> B -> C -> D


## Bellman-Ford

#### Theory

In [7]:
"""
Bellman-Ford Algorithm:
    - Computes the shortest paths from a single source vertex in a weighted graph.
    - Supports graphs with negative edge weights.
    - Detects negative-weight cycles.
    - Cannot compute shortest paths if a negative-weight cycle is reachable from the source.

Note:
    - Finding shortest paths in graphs with negative cycles is NP-hard problem.
    Reference: 
        - https://cstheory.stackexchange.com/questions/17462/finding-the-shortest-path-in-the-presence-of-negative-cycles 

    
links:
    - https://www.youtube.com/watch?v=j0OUwduDOS0
    - https://www.youtube.com/watch?v=SiI03wnREt4
    - https://www.youtube.com/watch?v=0hXj05XXT7M

"""

pass

#### Pseudo Code

In [8]:
"""
Pseudo code:
1. Initialization:
    - Set the distance of all vertices to infinity (∞), except the source vertex, which is set to 0.
    - Initialize the parent of all vertices as None.

2. Edge Relaxation:
    - Relax all edges |V-1| times, where |V| is the number of vertices, to calculate the shortest distances.
        +-----------------------------------------+
        | If d[u] + cost(u, v) < d[v]:            |
        |     Update d[v] = d[u] + cost(u, v)     |
        |     Update parent[v] = u                |
        +-----------------------------------------+

3. Negative Cycle Detection:
    - After performing |V-1| edge relaxations, try relaxing the edges one more time:
        - If any edge can still be relaxed, the graph contains a negative-weight cycle. (Raise an exception)

4. Return:
    - Return the shortest distances (d) and the parent dictionary representing the shortest path tree.


Complexity:
    Time: O(V * E), where V is the number of vertices and E is the number of edges.
        (Each edge is relaxed |V-1| times, plus one additional check for cycles.)
    Space: O(V + E) for storing distances and the graph structure.

"""

pass

#### Code

In [9]:
def bellman_ford(graph: dict, source):
    # Step 1: Initialization
    parent = {source: None}
    distance = {v: float("inf") for v in graph}
    distance[source] = 0

    # Step 2: Edge relaxation V-1 times
    for _ in range(len(graph) - 1):  # Complexity: |V -1| * E
        for vertex in graph:
            for adj_node, cost in graph[vertex]:
                if distance[vertex] + cost < distance[adj_node]:
                    distance[adj_node] = distance[vertex] + cost
                    parent[adj_node] = vertex

    # Step 3: Check for negative-weight cycles
    for vertex in graph:
        for adj_node, cost in graph[vertex]:
            if distance[vertex] + cost < distance[adj_node]:
                raise ValueError("Graph contains a negative-weight cycle")

    return distance, parent

In [10]:
graph = {
    "A": [("B", 1), ("D", 8)],
    "B": [("A", 1), ("C", 2), ("D", 6)],
    "C": [("B", 2), ("D", 3)],
    "D": [("A", 8), ("B", 6), ("C", 3)],
}

distance, route = bellman_ford(graph, "A")
print(f"{distance = }")
print(f"{route = }")

distance = {'A': 0, 'B': 1, 'C': 3, 'D': 6}
route = {'A': None, 'B': 'A', 'D': 'C', 'C': 'B'}


## Dijkstra vs. Bellman-ford